## Requirements

In [ ]:
!pip install --quiet pandas numpy

In [ ]:
import os
import json
import sqlite3
import random
import re
import ast
import csv
import pandas as pd
import numpy as np
import time

pd.set_option('display.width', 180)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
if not os.path.exists("first_names.csv"):
  !wget -O first_names.csv https://www.dropbox.com/scl/fi/lpybg77w95zkjunsrfnpn/first_names.csv?rlkey=tcajnzfhm11lotrmk429zznbu&dl=1

if not os.path.exists("demonyms.txt"):
  !wget -O demonyms.txt https://www.dropbox.com/scl/fi/uxt7c01ct0bijh7t0lbrw/demonyms.txt?rlkey=w6gnh1a073njpeolqe4tno6k3&dl=1

## Load Dictionary

In [ ]:
if not os.path.exists("hn_fr.db"):
  !wget -O hn_fr.db https://www.dropbox.com/scl/fi/319hl9s7rxmhpmkjbyq45/hn_fr.db?rlkey=91n6gjkh9rf9guhuqigx3lfcd&dl=1

if not os.path.exists("nhuma_full.csv"):
  !wget -O nhuma_full.csv https://www.dropbox.com/scl/fi/7uhmgd8she27k4tsvkwfe/nhuma.csv?rlkey=ghiyxvd09ki5mxevuies26uf2&dl=1

In [ ]:
conn = sqlite3.connect('hn_fr.db')

dmnt_df = pd.read_sql_query("SELECT * FROM dmnt", conn)
# keep demonyms capitalized
dmnt_df.update(dmnt_df.select_dtypes(include="object").apply(
    lambda col: col.where(dmnt_df["__dmnt__proper_noun"] == 1, col.str.lower())
))

wikt_df = pd.read_sql_query("SELECT * FROM wikt", conn).map(lambda x: x.lower() if isinstance(x, str) else x)

tlfi_df = pd.read_sql_query("SELECT * FROM tlfi", conn)
tlfi_cols_to_convert = [col for col in tlfi_df.columns if col != '__tlfi__completeWordForm']
tlfi_df[tlfi_cols_to_convert] = tlfi_df[tlfi_cols_to_convert].map(lambda x: x.lower() if isinstance(x, str) else x)
conn.close()

nhuma_df = pd.read_csv('nhuma_full.csv')

In [ ]:
# @title Remove Ambiguous Nouns

print("Before removing ambiguous nouns:", len(dmnt_df), len(tlfi_df), len(wikt_df), len(nhuma_df))

if not os.path.exists("animals.csv"):
  !wget -O animals.csv https://www.dropbox.com/scl/fi/vly91kjwxi1zhl8y15q63/animals.csv?rlkey=ck2q8urmybh1h9fceiz3l70et&dl=1

if not os.path.exists("objects.txt"):
  !wget -O objects.txt https://www.dropbox.com/scl/fi/wq7gz66urmwd4uu3ae5rb/objects.txt?rlkey=hpswx47e792fohktcii4zkcoe&dl=1

if not os.path.exists("others.txt"):
  !wget -O others.txt https://www.dropbox.com/scl/fi/ex8soy70l9e0kitafj7xe/others.txt?rlkey=ynsyy8jj2ngcl5kwzdv37rfss&dl=1

if not os.path.exists("colors.txt"):
  !wget -O colors.txt https://www.dropbox.com/scl/fi/s80ptb47tgdrpz6h83wse/colors.txt?rlkey=g1agxsgxr7kwkl7m795q3fu00&dl=1

animal_names = pd.read_csv("animals.csv")
animal_list = animal_names["animal"].tolist()

object_list =  open("objects.txt", "r").read().splitlines()

others_list = open("others.txt", "r").read().splitlines()

colors_list = open("colors.txt", "r").read().splitlines()

words_to_remove = animal_list + object_list + others_list + colors_list
words_to_remove = set(word.lower() for word in words_to_remove)

print("écho" in words_to_remove)

dmnt_df = dmnt_df[~dmnt_df["__dmnt__nm"].isin(words_to_remove)]
tlfi_df = tlfi_df[~tlfi_df["__tlfi__noun"].isin(words_to_remove)]
wikt_df = wikt_df[~wikt_df["__wikt__noun"].isin(words_to_remove)]
nhuma_df = nhuma_df[~nhuma_df["Lexème vedette"].isin(words_to_remove)]

print("After removing ambiguous nouns:", len(dmnt_df), len(tlfi_df), len(wikt_df), len(nhuma_df))

Before removing ambiguous nouns: 8047 7918 6686 1106
True
After removing ambiguous nouns: 7909 7573 6484 1093


In [ ]:
def get_item_number(json_string):
    try:
        json_data = json.loads(json_string)
        key = list(json_data.keys())[0]
        if key.endswith("item"): # handles primary keys (those without numbers)
            return 0
        number = int(key.split("item")[-1])
        return number
    except Exception as e:
        return None

print(len(wikt_df))

wikt_df['item_number'] = wikt_df['__wikt__senses_item_glosses_item'].apply(get_item_number)

wikt_df = wikt_df[wikt_df['item_number'] <= 2]

wikt_df = wikt_df.drop(columns=['item_number'])

print(len(wikt_df))

6484
5781


<ipython-input-11-1b95e444dcb3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wikt_df['item_number'] = wikt_df['__wikt__senses_item_glosses_item'].apply(get_item_number)


### Get Human List (M + F)

In [ ]:
def get_wikt_fem(wikt_df, excluded_f_words):
  feminine_forms = []

  for index, row in wikt_df.iterrows():
      forms_json = row[4]  # JSON string for forms
      tags_json = row[5]   # JSON string for tags

      forms = json.loads(forms_json)
      tags = json.loads(tags_json)

      # extract forms with 'feminine' tag
      for key, tag in tags.items():
          if "feminine" in tag:
              form_key = key.replace('tags', 'form')  # match the corresponding form key
              feminine_word = forms.get(form_key)

              if feminine_word[0] == feminine_word[0].lower() and feminine_word not in excluded_f_words:
                feminine_forms.append(feminine_word)

  conn.close()

  print(f"Retrieved {len(feminine_forms)} feminine nouns from Wiktionary")

  return feminine_forms

In [ ]:
neutrals = ["personne", "personnes", "individu", "individus", "gens"]
excluded_f_words = ["carrière", "colonne", "bienvenue", "boulette", "seconde", "teinte", "bordelière"] + list(words_to_remove)
add_words = ["protagoniste", "mentor", "mentore"]

demonyms = open("demonyms.txt", "r").read().splitlines()
ambiguous_demonyms = ["Poisson", "Poissons", "Gémeau", "Gémeaux", "Écho", "Échos", "Echo", "Echos"]

def get_human_list(dmnt_df, wikt_df, nhuma_df):
  print(len(wikt_df))

  wikt_df = wikt_df[~wikt_df['__wikt__senses_item_glosses_item'].str.contains('ensemble')]

  print(len(wikt_df))

  wikt_list = wikt_df['__wikt__noun'].tolist() + get_wikt_fem(wikt_df, excluded_f_words)

  print(len(dmnt_df['__dmnt__nm'].tolist()), len(dmnt_df['__dmnt__nf'].tolist()))

  dmnt_df_nf = dmnt_df['__dmnt__nf'][~dmnt_df['__dmnt__nf'].isin(excluded_f_words)]
  dmnt_list = dmnt_df['__dmnt__nm'].tolist() + dmnt_df_nf.tolist()

  nhuma_list = nhuma_df['Lexème vedette'].tolist()

  print(f"Wiktionary list total length: {len(wikt_list)} human nouns")
  print(f"Dmnt list total length: {len(dmnt_list)} human nouns")
  print(f"NHUMA list total length: {len(nhuma_list)} human nouns")
  print(f"Demonyms list total length: {len(demonyms)} human nouns")
  print(f"Neutral list total length: {len(neutrals)} human nouns")

  print("écho" in wikt_list, "écho" in dmnt_list, "écho" in nhuma_list)

  print(nhuma_list)

  human_list = wikt_list + dmnt_list + nhuma_list + neutrals + add_words

  human_list = [s for s in human_list if s.capitalize() not in demonyms and s not in ambiguous_demonyms]

  human_list += demonyms
  human_list = set(human_list)

  conn.close()

  print(f"Human list total length: {len(human_list)} human nouns")

  return human_list

In [ ]:
human_list = get_human_list(dmnt_df, wikt_df, nhuma_df)

5781
5776


<ipython-input-12-245a5696b196>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  forms_json = row[4]  # JSON string for forms
<ipython-input-12-245a5696b196>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tags_json = row[5]   # JSON string for tags


Retrieved 2173 feminine nouns from Wiktionary
7909 7909
Wiktionary list total length: 7949 human nouns
Dmnt list total length: 15798 human nouns
NHUMA list total length: 1093 human nouns
Demonyms list total length: 413 human nouns
Neutral list total length: 5 human nouns
False False False
['abandonnataire', 'abandonneur', 'abbé', 'abonnataire', 'aboyeur', 'abreuveur', 'abréviateur', 'abstinent', 'abuseur', 'académiste', 'accepteur', 'accessoiriste', 'accidenté', 'acclamateur', 'accomodeur', 'accompagnateur', 'accompagnateur de moyenne montagne', 'accompagnateur de tourisme', 'accompagnateur de tourisme équestre', 'accomplisseur', 'acconier', 'accordéoniste', 'accordeur', 'accouchée', 'accoucheur', 'accusateur', 'accusé', 'acheteur', "acheteur d'espaces publicitaires", 'acheteur de composants', 'acheteur industriel', 'acheteur informatique et télécom', 'acheteur-approvisionneur', 'acheteur-vendeur', 'acolyte', 'acousticien', 'acquéreur', 'acteur', 'actionnaire', 'actuaire', 'acupuncteur

In [ ]:
"écho" in human_list

False

In [ ]:
human_df = pd.DataFrame(human_list, columns=['noun'])

In [ ]:
human_df.to_pickle("human_df.pkl")

### Get Epicene

In [ ]:
epicene_list = dmnt_df.loc[(dmnt_df['__dmnt__epicene'] == 1) & (dmnt_df['__dmnt__proper_noun'] == 0), '__dmnt__nm'].tolist()

### Merge DMNT and NHUMA (get human noun classes)

In [ ]:
dmnt_df = dmnt_df.loc[(dmnt_df['__dmnt__epicene'] == 0) & (dmnt_df['__dmnt__proper_noun'] == 0), '__dmnt__nm'].to_frame()

In [ ]:
nhuma_df['Lexème vedette'] = nhuma_df['Lexème vedette'].str.lower()
dmnt_df['__dmnt__nm'] = dmnt_df['__dmnt__nm'].str.lower()

dmnt_df = dmnt_df.merge(nhuma_df[['Lexème vedette', 'Classe sémantique', 'Sous-classe sémantique']],
                          left_on='__dmnt__nm', right_on='Lexème vedette',
                          how='left').fillna("")
dmnt_df = dmnt_df.drop(columns=['Lexème vedette'])

In [ ]:
dmnt_df

,__dmnt__nm,Classe sémantique,Sous-classe sémantique
0,abandonnateur,,
1,abandonneur,N d'Agent,
2,abatteur,,
3,abbé,N de Statut,
4,abonné,,
...,...,...,...
5023,mal-pensant,,
5024,blousard,,
5025,czar,,
5026,protéomicien,,


### Merge TLFI and NHUMA (get human noun classes)

In [ ]:
tlfi_df['__tlfi__noun'] = tlfi_df['__tlfi__noun'].str.lower()

tlfi_df = tlfi_df.loc[
    tlfi_df['__tlfi__wordType'].str.contains('subst', na=False) &
    tlfi_df['__tlfi__completeWordForm'].str.match(r'^[A-Z]+,\s[A-Z]+') &
    tlfi_df['__tlfi__completeWordForm'].apply(
        lambda x: len(x.split(',')[1].strip()) < len(x.split(',')[0].strip())-1
    ),
    '__tlfi__noun'
].to_frame()

In [ ]:
tlfi_df = tlfi_df.merge(nhuma_df[['Lexème vedette', 'Classe sémantique', 'Sous-classe sémantique']],
                          left_on='__tlfi__noun', right_on='Lexème vedette',
                          how='left').fillna("")
tlfi_df = tlfi_df.drop(columns=['Lexème vedette'])

In [ ]:
tlfi_df

,__tlfi__noun,Classe sémantique,Sous-classe sémantique
0,abruti,,
1,absent,,
2,abstinent,Autre,
3,accompagnateur,N d'Agent,
4,accouchant,,
...,...,...,...
738,aquarien,,
739,boudinier,,
740,contadin,,
741,ausonien,,


### Get Final Masculine Generics DF

In [ ]:
tlfi_df = tlfi_df.rename(columns={'__tlfi__noun': 'noun'})
dmnt_df = dmnt_df.rename(columns={'__dmnt__nm': 'noun'})
human_df = pd.merge(tlfi_df, dmnt_df,
                    on='noun',
                    how='outer',
                    suffixes=('_tlfi', '_dmnt'))

human_df['Classe sémantique'] = human_df['Classe sémantique_tlfi'].combine_first(human_df['Classe sémantique_dmnt'])
human_df['Sous-classe sémantique'] = human_df['Sous-classe sémantique_tlfi'].combine_first(human_df['Sous-classe sémantique_dmnt'])

human_df = human_df.drop(columns=['Classe sémantique_tlfi', 'Classe sémantique_dmnt',
                                  'Sous-classe sémantique_tlfi', 'Sous-classe sémantique_dmnt']).drop_duplicates(["noun"], ignore_index=True)

masc_gen_df = human_df.rename(columns={'Classe sémantique': 'class', 'Sous-classe sémantique': 'subclass'})

#### add demonyms/remove specific epicenes

In [ ]:
epicenes = ["médecin"]
demonym_suffixes = ["ais", "ois", "ain", "ien", "in", "an", "en", "ol", "and", "ec", "on", "ard", "urc"]
masc_gen_demonyms = [s for s in demonyms if s.endswith(tuple(demonym_suffixes))]

# add to masc_gen_df with class N de Population (Gentilé), subclass Pays
demonym_entries = pd.DataFrame({
    'noun': masc_gen_demonyms,
    'class': 'N de Population (Gentilé)',
    'subclass': 'Pays'
})

masc_gen_df = masc_gen_df[~masc_gen_df['noun'].isin(epicenes)]
epicene_list += epicenes

masc_gen_df = pd.concat([masc_gen_df, demonym_entries], ignore_index=True)

In [ ]:
masc_gen_df

,noun,class,subclass
0,abruti,,
1,absent,,
2,abstinent,Autre,
3,accompagnateur,N d'Agent,
4,accouchant,,
...,...,...,...
5140,Wallisien,N de Population (Gentilé),Pays
5141,Futunien,N de Population (Gentilé),Pays
5142,Wallon,N de Population (Gentilé),Pays
5143,Zambien,N de Population (Gentilé),Pays


#### apply_model_barbara

In [ ]:
from spacy.tokens import DocBin
from spacy import displacy
import string
nlp_nh = spacy.load(R"masc_gen_paper/nh_model")

h_words = {line.rstrip() for line in open("masc_gen_paper/nh_model/h_wiktionary.txt")}

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'fr_pipeline' (0.0.0) was trained with spaCy v3.5.3 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
missing_classes_before_annot = len(masc_gen_df[masc_gen_df["class"].isna() | (masc_gen_df["class"] == "")])
print(len(masc_gen_df), missing_classes_before_annot)

5145 4356


In [ ]:
def get_human_label(text, nlp=nlp_nh):
    #colors = {"NH-Fonc": "#aa9cfc", "NH-Mét": "#fc9ce7", "NH-Spé": "#f5426c", "NH-Grade": "#7bf542", "NH-Titre": "#7bf542"}
    #options = {"ents": ["NH-Fonc", "NH-Mét", "NH-Spé", "NH-Grade", "NH-Titre"], "colors": colors}
    doc = nlp(text)
    #print(f"Found entity in text: {doc.ents}")
    for ent in doc.ents:
        return ent.label_
    else:
        return ""
    #displacy.render(doc, style="ent", options=options, jupyter=True)

In [ ]:
def starts_vowel(word, h_words=h_words):
    vowels = "aâéeèiïîoôöuûüAÂÉEÈIÏÎOÔÖUÛÜ"

    if (word[0] in vowels) or (word[0] == "h" and word.strip(string.punctuation) not in h_words):
        return True

    return False

def create_generic_text(human_noun):
    if starts_vowel(human_noun):
        det = "l'"
    else:
        det = "le"
    return f"Toute sa vie durant, {det} {human_noun} n'a fait qu'essuyer les échecs."

def append_class(row):
    noun = row["noun"]

    generic_text = create_generic_text(noun)
    label = get_human_label(generic_text)

    # if no label is found, try with only the noun
    if label == "":
        label = get_human_label(noun)

    return label

masc_gen_wo_classes = masc_gen_df[masc_gen_df["class"].isna() | (masc_gen_df["class"] == "")]

masc_gen_wo_classes["class"] = masc_gen_wo_classes.apply(append_class, axis=1)

<ipython-input-31-74c74d75ba44>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  masc_gen_wo_classes["class"] = masc_gen_wo_classes.apply(append_class, axis=1)


In [ ]:
# final df with class annotation from the model
masc_gen_df = masc_gen_df.set_index("noun")
masc_gen_wo_classes = masc_gen_wo_classes.set_index("noun")

masc_gen_df = masc_gen_wo_classes.combine_first(masc_gen_df).reset_index()

# masc_gen_df = pd.concat([masc_gen_df, masc_gen_wo_classes], ignore_index=True)
missing_classes_after_annot = len(masc_gen_df[masc_gen_df["class"].isna() | (masc_gen_df["class"] == "")])

print(f"Missing classes before annotation: {missing_classes_before_annot} ({missing_classes_before_annot/len(masc_gen_df)*100:.2f}%)")
print(f"Missing classes after annotation: {missing_classes_after_annot} ({missing_classes_after_annot/len(masc_gen_df)*100:.2f}%)")

Missing classes before annotation: 4356 (84.66%)
Missing classes after annotation: 2873 (55.84%)


#### class/subclass translations & merge

In [ ]:
unique_classes = masc_gen_df['class'].unique()
unique_subclasses = masc_gen_df['subclass'].unique()

print("Unique classes:", unique_classes)
print("Unique subclasses:", unique_subclasses)

Unique classes: ['N de Population (Gentilé)' '' "N d'Agent" 'NH-Mét' 'N de Statut' 'Autre'
 'N de Patient/Déverbal passif' 'NH-Spé' 'NH-Fonc'
 'N d’Adepte/de Partisan' 'N Relationnel ' 'N de Caractérisant'
 'N d’Habitant' 'NH-Grade' 'NH-Titre' 'N d’Âge' 'N de Bénéficiaire'
 'N Général' 'N de Possesseur']
Unique subclasses: ['Pays' '' 'N de Profession ' 'Membre' 'N de Pratique ' ' Pays'
 'N de Créateur' 'N de Fonction' 'N de Spécialité' 'Région ' 'Parenté'
 ' Ville' 'Grade' 'N de Défaut moral' 'N d’Interprète'
 'N de Qualité morale' 'Titre' 'Supranational' 'Société']


In [ ]:
# strip leading spaces
masc_gen_df["class"] = masc_gen_df["class"].str.strip()
masc_gen_df["subclass"] = masc_gen_df["subclass"].str.strip()

# translate
class_translations = {
    "N de Population (Gentilé)": "demonym",
    "N d'Agent": "doer", # does smth; to merge w/ créateur subclass?
    "NH-Mét": "profession",
    "N de Statut": "status",
    "Autre": "other",
    "N de Patient/Déverbal passif": "patient", #mergeable with "attribute"?
    "NH-Spé": "speciality",
    "NH-Fonc": "profession", # same as NH-Mét?
    "N d’Adepte/de Partisan": "", # ???
    "N Relationnel": "relationship", # ?
    "N de Caractérisant": "attribute", # ? aveugle, défauts / fou / défaut psychologique/psychique
    "N d’Habitant": "demonym",
    "NH-Grade": "title", # mergeable?
    "NH-Titre": "title", # mergeable?
    "N d’Âge": "", # ??? "gynécologue"
    "N de Bénéficiaire": "recipient",
    "N Général": "", # ???
    "N de Possesseur": "", # ???
}

#note: wouldn't it be preferable for subclasses to be the same as
#classes in order to have coherent groups of classes
#and create a new column with these classes,
#even if that means having several classes for one word?
subclass_translations = {
    "Pays": "demonym",
    "N de Profession": "profession", # diff with NH-Mét/NH-Fonc?
    "Membre": "",
    "N de Pratique": "",
    "N de Créateur": "doer",
    "N de Fonction": "profession",
    "N de Spécialité": "speciality",
    "Région": "demonym",
    "Parenté": "relationship",
    "Ville": "demonym",
    "Grade": "title",
    "N de Défaut moral": "attribute",
    "N d’Interprète": "",
    "N de Qualité morale": "", # p-e attribute
    "Titre": "title",
    "Supranational": "",
    "Société": "",
}

masc_gen_df["class"] = masc_gen_df["class"].replace(class_translations)
masc_gen_df["subclass"] = masc_gen_df["subclass"].replace(subclass_translations)

masc_gen_df["merged_classes"] = masc_gen_df.apply(
    lambda row: ", ".join(filter(None, {row["class"], row["subclass"]})), axis=1
)

In [ ]:
masc_gen_list = list(masc_gen_df["noun"])

In [ ]:
masc_gen_df.to_pickle("masc_gen_df.pkl")